In [ ]:
def split_sentences(text):
    """
    return a list of sentences from text based on common separators
    """
    sentence_delimiters = re.compile(u'[.!?,;:\t\\\\"\\(\\)\\\'\u2019\u2013]|\\s\\-\\s')
    sentences = sentence_delimiters.split(text)
    
    return sentences

test = split_sentences(test)
#test

In [ ]:
def build_stop_word_regex():
    """
    :used in function generate_phrases to remove stopwords before
    compiling multi word phrases
    """
    stop_word_list = spacy.lang.en.stop_words.STOP_WORDS
    stop_word_regex_list = []
    for word in stop_word_list:
        word_regex = r'\b' + word + r'(?![\w-])'
        stop_word_regex_list.append(word_regex)
    stop_word_pattern = re.compile('|'.join(stop_word_regex_list), re.IGNORECASE)
    return stop_word_pattern

def generate_phrases(text, stopword_regex):
    """
    :compile list of multi word phrases by splitting sentences on stopwords
    """
    
    phrase_list = []
    for sentence in text:
        # replace stopwords with | in order to break sentences up
        tmp = re.sub(stopword_regex, '|', sentence.strip())
        phrases = tmp.split("|")
        for phrase in phrases:
            phrase = phrase.strip().lower()
            phrase = ' '.join(phrase.split())
            if phrase != "":
                phrase_list.append(phrase)
    phrase_list = [phrase for phrase in phrase_list if re.search(r'\w+', phrase)]
    
    return phrase_list

def remove_starting_symbol(phrase_list):
    """
    :remove starting symbols or numbers from phrases e.g '“whilst rewearing' or '-based website'
    """
    phrase_list_cleaned = []
    
    for phrase in phrase_list:
        try:
            while not phrase[0].isalpha():
                phrase = phrase[1:]
        except IndexError:
            pass
        phrase_list_cleaned.append(phrase)
            
    return phrase_list_cleaned

def remove_single_character_phrase(phrase_list):
    
    phrase_list_cleaned = [phrase for phrase in phrase_list if len(phrase) > 1]
            
    return phrase_list_cleaned


# Combined function of above
def find_multi_word_terms(df):
    df = df.copy()
    df['mwt'] = ''
    
    stopword_regex = build_stop_word_regex()
    
    for idx, content in enumerate(df['content_processed']):
        text = generate_phrases(text, stopword_regex)
        text = remove_starting_symbol(text)
        #print('PHRASES: \n', text, '\n')
        text = remove_single_character_phrase(text)
        df_new.at[idx, 'content_processed'] = text
        if idx % 25000 == 0:
            print(f'{idx} records processed')
    
    return df_new

In [ ]:
stopword_regex = build_stop_word_regex()
test = generate_phrases(test, stopword_regex)
#test